You need to make sure you have tika and get the docker image running with this command: 

IMAGE=tgowda/rtg-model:500toEng-v1

docker run --platform linux/amd64 --rm -i -p 6060:6060 $IMAGE

I added in  --platform Linux/amd64  to the command bc I’m on a mac and I needed that to run the image 

You’ll also need to pip install tika and emoji. I accidentally installed / imported requests too but didn’t end up using it 

Please make sure you have Java1.8 or higher intsalled on the environment you are using to run the notebook

In [99]:
import pandas as pd
from tika import translate
from tika import language
import requests
import re
import emoji

In [ ]:
df = pd.read_csv('../Datasets/Master_Dataset_Raw_LangDetect.csv')

In [105]:
df['Narrative'] = df['Narrative'].astype(str)

In [ ]:
#taken from the emoji docs since it's not in the latest versions
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

exp = get_emoji_regexp()

In [106]:
#this function removes any emojis, html tags, URLs, and special characters before translating
def clean_text(text):
    try:
        text = exp.sub(u'', text)
        text = re.sub(r'<.*?>', '', text)
        text = re.sub(r'http\S+', '', text)
        text = re.sub('[^a-zA-Z]+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    except :
        print(text)
    finally:
        return text

0        The colour of the year is here and it s drumro...
1        We Indians do love to bastardise our foods Chi...
2        Can professors not have personal lives Additio...
3        She was recorded on video manhandling him shou...
4        The House passed the Inflation Reduction Act o...
                               ...                        
94995                                           MODR CLASS
94996    College life without a friend is useless Enjoy...
94997               Sheridan college HazelMcCallion Canpus
94998                                      Check this out 
94999    Spending time in college with these two maddie...
Name: Narrative, Length: 95000, dtype: object


In [ ]:
clean_narr = clean_text(df['Narrative'].apply(clean_text))

In [ ]:
df['Clean Narrative'] = clean_narr

In [ ]:
chunks = 2 #can increase this number, my computer couldn't even handle 2 words per translation

translated_column = []
for text, language in zip(df['Clean Narrative'], df['Narrative TikaDetect']):
    if language != 'en':
        words = text.split(" ")
        #translate the narrative in chunks depending on the limit set and join them all together at the end
        if len(words) > chunks:
            translated_chunks = []
            try:
                for i in range(0, len(words), chunks):
                    group = ' '.join(words[i:i+chunks])
                    translation = translate.from_buffer(group, language, 'en', requestOptions={'timeout':500})
                    translated_chunks.append(translation)
                translated_text = ' '.join(translated_chunks)
                translated_column.append(translated_text)
            except:
                print(text)
            finally:
                #Appending "redo" for narratives that fail so they will be searchable to run again and update
                translated_column.append('redo')
        #translate the entire narrative if it's under the specified chunk size
        else:
            try:
                translated_text = translate.from_buffer(text, language, 'en', requestOptions={'timeout':500})
                translated_column.append(translated_text)
            except:
                print(text)
            finally:
                #Appending "redo" for narratives that fail so they will be searchable to run again and update
                translated_column.append('redo')
    else:
        #just append the regular text if it's already english
        translated_column.append(text)

In [ ]:
df['Translated Narrative'] = translated_column

In [ ]:
df.to_csv('Master_Dataset_Raw_LangDetect_Translated.csv', index=False)